In [1]:
import itertools
import numpy as np
import random
from random import sample
from random import choices
def findsubsets(S,m):
    return set(itertools.combinations(S, m))

In [2]:
def preferences_of_courses(n_of_c, n_of_s): #случайно генерирует предпочтения курсов
    preferences = list()
    for i in range(n_of_c):
        preferences.append(np.random.choice(np.arange(n_of_s), size = n_of_s, replace= False))
    return preferences

In [3]:
def allsubsets(S):
    a = []
    for m in range(2,len(S)):
        a.extend(findsubsets(S,m))
    return a

In [4]:
def feasibility(x, b, credits): #проверяет допустим ли набор курсов
    price = 0
    if x is not None:
        for i in x:
            price = price + credits[i]
        return price == b
    else: return False

In [5]:
def feasible_subsets(courses, b):
    a = allsubsets(courses)
    feasible = list()
    for i in a:
        if feasibility(i,b,credits):
            feasible.append(i)
    return feasible

In [6]:
def preferences_of_students(n_of_s, feas_sub): #список списков кортежей
    preferences = list()
    for i in range(n_of_s):
        preferences.append(sample(feas_sub,len(feas_sub)))
    return preferences

In [7]:
def feasible_after(a, agreed): #ищет допустимые курсы при условии, что часть уже приняла
    #a - предпочтения студента (список кортежей), agreed - принявшие курсы(список)
    a = a.copy()
    output = []
    if agreed is not None:
        agreed = agreed.copy()
        agreed = set(agreed)
        for i in a:
            i = set(i)
            if i.intersection(agreed) == agreed and i != agreed:
                output.append(tuple(i - agreed))
        return output
    else: return a

In [8]:
def feasible_after_disagreement(a, disagred): #ищет допустиые наборы при условии, что часть отвергла
    a = a.copy()
    output = list()
    if disagred is not None: #если множество не пусто
        disagred = disagred.copy()
        disagred = set(disagred)  
        for i in a:
            i = set(i)
            if i.intersection(disagred) == set():
                output.append(tuple(i))
        return output
    else:
        return a

In [9]:
def search_for_worst(admitted, candidate, preferences): #выдает худшего студента среди отправленных
    #preferences - массив
    preferences = np.copy(preferences)
    preferences = list(preferences)
    worst = admitted[0]
    for i in admitted:
        if preferences.index(i) > preferences.index(worst):
            worst = i
    if preferences.index(worst) > preferences.index(candidate):
        return worst
    else: return candidate

In [10]:
#на вход - список курсов, список весов, список студентов, список квот курсов,
#массивы предпочтений студентов и курсов
def algorithm(courses, credits, students, quota, b, pref_c, pref_s):
    quota = np.copy(quota)
    students_iter = itertools.cycle(students)
    iterations = 0
    pref_s_start = pref_s.copy() #предпочтения студентов до модификаций
    pref_s = pref_s.copy()
    disagred_s = np.empty(shape = len(students),dtype= list) #список курсов, которые отказали студенту
    admitted_students = np.empty(shape = len(courses),dtype = list) #списки принятых студентов
    admitted_courses = np.empty(shape = len(students), dtype = list)#списки принятых курсов
    for student in students_iter:
        #print('student', student)
        if len(pref_s[student]) != 0: #проверяем не пусты ли предпочтения
            iterations = 0 # если не пусты запускаем процесс заново
            best = pref_s[student][0] #лучший набор студента
            for i in best:
                if admitted_students[i] != None: #есть пренятые студенты 
                    if quota[i] - len(admitted_students[i]) != 0: #проверяем выбрана ли квота курса
                        admitted_students[i].append(student) #если не выбрана - просто добавляем студента
                        if admitted_courses[student] != None: 
                            admitted_courses[student].append(i)
                        else:
                            admitted_courses[student] = [i]
                    else: #квота выбрана
                        worst = search_for_worst(admitted_students[i], student, pref_c[i])#ищем худшего
                        if worst != student: #если студент не худший
                            admitted_students[i].remove(worst) #удаляем худшего из принятых студентов
                            admitted_courses[worst].remove(i) #удаляем курс у худшего студента
                            admitted_students[i].append(student)#добавляем нашего студента в принятые
                            if admitted_courses[student] != None: 
                                admitted_courses[student].append(i) #добавляем курс нашему студенту
                            else: admitted_courses[student] = [i] #добавляем курс нашему студенту
                            if disagred_s[worst] != None: #добавляем худшему отказавший курс
                                disagred_s[worst].append(i)
                            else:
                                disagred_s[worst] = [i]
                        else: #если студент худший
                            if disagred_s[student] != None:
                                disagred_s[student].append(i)
                            else:
                                disagred_s[student] = [i]
                else:#нет принятых студентов, добавляем нашего
                    admitted_students[i] = [student]
                    if admitted_courses[student] != None:
                        admitted_courses[student].append(i)
                    else:
                        admitted_courses[student] = [i]
        else:# предпочтения пусты
            iterations += 1        
        pref_s[student] = feasible_after(pref_s_start[student], admitted_courses[student])
        pref_s[student] = feasible_after_disagreement(pref_s[student], disagred_s[student])
        if len(pref_s[student]) > 0: iterations = 0
        if len(pref_s[student]) == 0 and not feasibility(admitted_courses[student], b, credits):
            min_quota = disagred_s[student][0]
            for dis in disagred_s[student]:
                if quota[dis] < quota[min_quota]:
                    min_quota = dis
            quota[min_quota] += 1
            return algorithm(courses, credits, students, quota, b, pref_c, pref_s_start)
        if iterations == len(students):
            return (admitted_students, admitted_courses, quota)
#на выход - массив принятых студентов (относительно курсов), массив принятых курсов(относительно студентов, вектор квот 